In [2]:
import json
import torch
import torch.nn as nn
from torch.autograd import Variable as Var
import torch.optim as optim

from syft.core.hooks import TorchHook
from syft.core.workers import VirtualWorker, SocketWorker

hook = TorchHook(local_worker=SocketWorker(id=1, port=8205, is_pointer=False, is_client_worker=False))

Starting Socket Worker...
Ready to receive commands...
Hooking into Torch...
Overloading complete.


In [3]:
remote_worker_metadata={"id": 1,
                        "type":"syft.core.workers.socket.SocketWorker",
                        "hostname":"localhost",
                        "port":8186}

def init_worker_pointer(hook, metadata):
    
    if(metadata['type'] == "syft.core.workers.socket.SocketWorker"):
        remote_client = SocketWorker(hook=hook,
                                     id=metadata['id'],
#                                      hostname=metadata['hostname'],
                                     port=metadata['port'],
                                     is_pointer=True)
        return remote_client
remote_client = init_worker_pointer(hook, remote_worker_metadata)

Attaching Pointer to Socket Worker...


In [4]:
data = Var(torch.FloatTensor([[0,0],[0,1]]))

In [5]:
data.id

1132951301

In [6]:
data.send(remote_client, send_pointer=True)

Variable containing:
 0  0
 0  1
[torch.FloatTensor of size 2x2]

In [6]:
hook.local_worker._objects[data.id]

Variable containing:
 0  0
 0  1
[torch.FloatTensor of size 2x2]

In [7]:
hook.local_worker.listen(1)

Received Command From: ('127.0.0.1', 58601)


In [4]:
# data.data.id

In [5]:
data.ser(include_data=False)

'{"torch_type": "torch.autograd.variable.Variable", "requires_grad": false, "volatile": false, "data": "{\\"torch_type\\": \\"torch.FloatTensor\\", \\"id\\": 9578980310, \\"owners\\": [1], \\"is_pointer\\": true}\\n", "grad": null, "id": 823384466, "owners": [1], "is_pointer": true}'

In [ ]:
data.send()

In [ ]:
hook.local_worker._listen()

Received Command From: ('127.0.0.1', 55299)


In [4]:
# FEDERATED LEARNING CLIENT
# eventually this would be some kind of browser application
# but for now we're going to prototype it in python

# 1) Initialize A Dataset (and register it with the local SocketWorker)
# 2) Tell the server about the dataset (i have a data and target tensor with ID x and y)
# 3) tell SocketWorker to listen for instructions (aka... how to do the federated learning part)

# Step 1: Initialize a Dataset

(and register it with the local SocketWorker)

In [5]:
data = Var(torch.FloatTensor([[0,0],[0,1]]))
target = Var(torch.FloatTensor([[0],[0]]))

In [6]:
dataset = {}
dataset['input'] = data.id
dataset['target'] = target.id

worker = hook.local_worker.whoami()

message_obj = {"dataset":dataset, "worker":worker}

In [8]:
type(hook.local_worker)

syft.core.workers.socket.SocketWorker

In [7]:
worker

'{"hostname": "localhost", "port": 8194, "id": 1}'

In [6]:
import asyncio
import websockets

async def hello():
    async with websockets.connect(
            'ws://localhost:8766') as websocket:
        message = json.dumps(message_obj)

        await websocket.send(message)
        print(f"> {message}")

        response = await websocket.recv()
        print(f"< {response}")

asyncio.get_event_loop().run_until_complete(hello())

> {"dataset": {"input": 5646722488, "target": 6328116976}, "worker": "{\"hostname\": \"localhost\", \"port\": 8184, \"id\": 0}"}
< Hello {'dataset': {'input': 5646722488, 'target': 6328116976}, 'worker': '{"hostname": "localhost", "port": 8184, "id": 0}'}!
